In [2]:
!pip install sodapy

In [3]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import  datetime as dt
dest = '/content/drive/MyDrive/Python/Chicago-Business/'
client = Socrata("data.cityofchicago.org", None)
results = client.get_all("r5kz-chrr",where="date_extract_y(date_issued) > 2009")
results_df = pd.DataFrame.from_records(results)
# transform to dataframe
results_df['date_issued'] = pd.to_datetime(results_df['date_issued'])
results_df['YearIssue'] = pd.DatetimeIndex(results_df['date_issued']).year

In [4]:
IssueDf = results_df[results_df['application_type']=='ISSUE']
IssueDf = IssueDf.drop(columns=['legal_name','doing_business_as_name','address','application_type','business_activity_id', 'business_activity'])
IssueDf['Month'] = IssueDf['date_issued'].dt.to_period('M')
IssueMon = pd.DataFrame(IssueDf.groupby('Month')['date_issued'].count())
IssueMon = IssueMon.rename(columns={"date_issued": "Count_License"})

In [6]:
#=========================#
#-- Incentive programms --#
#=========================#
# the incentives file is not big so just download it using the link in appendices
SBIF = pd.read_csv(dest + 'Incentive_SBIF.csv')
SBIF['APPROVAL DATE'] = pd.to_datetime(SBIF['APPROVAL DATE'])
SBIF['YearApproval'] = pd.DatetimeIndex(SBIF['APPROVAL DATE']).year
SBIF['Month'] = SBIF['APPROVAL DATE'].dt.to_period('M')
SBIFMon = pd.DataFrame(SBIF.groupby('Month')['INCENTIVE AMOUNT'].sum())
SBIFMon = SBIFMon.rename(columns={"INCENTIVE AMOUNT": "Sum_Incentive"})
SBIFMon=SBIFMon.reset_index()

In [7]:
#=========================#
#-- economic indicators --#
#=========================#
# download the files using the links in appendices
HPI = pd.read_csv(dest + 'CHXRSA.csv') # S&P/Case-Shiller Home Price Index
HPI['Month'] = pd.to_datetime(HPI['DATE']).dt.to_period('M')
HPI = HPI.drop(columns=['DATE'])
Emp = pd.read_csv(dest + 'CHIC917URN.csv') # Unemployment Rate
Emp['Month'] = pd.to_datetime(Emp['DATE']).dt.to_period('M')
Emp = Emp.drop(columns=['DATE'])
ElecPri = pd.read_csv(dest + 'APUS23A72610.csv') # Average Price: Electricity
ElecPri['Month'] = pd.to_datetime(ElecPri['DATE']).dt.to_period('M')
ElecPri = ElecPri.drop(columns=['DATE'])

In [24]:
#===============#
#-- merge all --#
#===============#
FullDat = IssueMon.merge(SBIFMon,how='left',on='Month')
FullDat = FullDat.merge(HPI,how='left',on='Month')
FullDat = FullDat.merge(Emp,how='left',on='Month')
FullDat = FullDat.merge(ElecPri,how='left',on='Month')
# FullDat.to_csv(dest + 'FullDat.csv')

In [ ]:
#=======================#
#-- log transformation--#
#=======================#
FullDat = FullDat[FullDat['Month']<'2021-09']
# log transformation
FullDat['Log_Count_License'] = np.log(FullDat['Count_License'])
FullDat['Log_Sum_Incentive'] = np.log(FullDat['Sum_Incentive'])
FullDat['Log_HPI'] = np.log(FullDat['CHXRSA'])
FullDat['Log_Emp'] = np.log(FullDat['CHIC917URN'])
FullDat['APUS23A72610'] = FullDat['APUS23A72610'].astype(float)
FullDat['Log_Elec'] = np.log(FullDat['APUS23A72610'])
FullDat = FullDat.drop(columns=['Count_License','Sum_Incentive','CHXRSA','CHIC917URN','APUS23A72610'])

In [26]:
# replace empty incentive cells in 2020 with 0's
FullDat = FullDat.replace(np.nan, 0)
# calculate lags at t-3
FullDat['Log_Sum_Incentive_lag3']=FullDat['Log_Sum_Incentive'].shift(3)
FullDat['Log_HPI_lag3']=FullDat['Log_HPI'].shift(3)
FullDat['Log_Emp_lag3']=FullDat['Log_Emp'].shift(3)
FullDat['Log_Elec_lag3']=FullDat['Log_Elec'].shift(3)
# calculate lags at t-6
FullDat['Log_Sum_Incentive_lag6']=FullDat['Log_Sum_Incentive'].shift(6)
FullDat['Log_HPI_lag6']=FullDat['Log_HPI'].shift(6)
FullDat['Log_Emp_lag6']=FullDat['Log_Emp'].shift(6)
FullDat['Log_Elec_lag6']=FullDat['Log_Elec'].shift(6)
FullDat.to_csv(dest + 'FullDat-2.csv',index=False)